We now look to model forward returns with our four factor styles, `MOMENTUM`, `GROWTH`, `VALUE` and `SIZE`.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import yaml
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from sklearn.linear_model import LinearRegression
import seaborn as sns

load_dotenv()

# .env variables
db_url = os.getenv('DB_URL')
constituents_table = os.getenv('CONSTITUENTS_TABLE')
prices_table = os.getenv('PRICES_TABLE')

# connection to sql database
psql = create_engine(db_url)

In [2]:
adjusted_monthly = pd.read_sql("""
    select
        *
    from
        adjusted_monthly
    """,
    psql
)

adjusted_monthly['adj_return'] = np.log(adjusted_monthly['adj_close'] / adjusted_monthly['adj_close'].shift())
adjusted_monthly['adj_return_fwd'] = adjusted_monthly['adj_return'].shift(-1)
adjusted_monthly = adjusted_monthly.dropna()

In [11]:
factor_composites = pd.read_sql("""
    select * from factor_composites
""", psql).drop_duplicates()

wide_factor_data = (
    factor_composites
        .pivot(index=['date', 'ticker'], columns='style', values='score')
        .reset_index()
        .dropna()
)

In [13]:
monthly_factor = adjusted_monthly.merge(wide_factor_data, on=['date', 'ticker'])